In [ ]:
import numpy as np
import os  # OS library can be used for reading folders,sub-folders or files inside a directory.
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm  # Can be used as a progress tracker

In [ ]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

print (class_names_label)

number_of_classes = len(class_names)

IMAGE_SIZE = (150, 150)

In [ ]:
def load_data():
    datasets = [r'C:\Users\lenovo\Desktop\ExcelR\AI Images\seg_train\seg_train',
                r'C:\Users\lenovo\Desktop\ExcelR\AI Images\seg_test\seg_test']
    output = []
    
    #Iterating through Train & Test Data Set.
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading Started for ",dataset)
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):   # 6 times for each folder
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            # Example Path: ../Data/seg_train/seg_train/buildings
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
print (train_labels.shape)
print (test_labels.shape)
print (train_labels)

In [ ]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

In [ ]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [ ]:
index = np.random.randint(train_images.shape[0])
plt.figure()
plt.imshow(train_images[index])
plt.grid(False)
plt.title('Image #{} : '.format(index) + class_names[train_labels[index]])
plt.show()

In [ ]:
model_scratch = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(84, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

model_scratch.summary()

In [ ]:
model_scratch.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model_scratch.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.15)

In [ ]:
for layer in model_scratch.layers:
    print (layer.get_weights())

In [ ]:
test_loss = model_scratch.evaluate(test_images, test_labels)

In [ ]:
image = cv2.imread("../Data/Evaluate/stre_1.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, IMAGE_SIZE)
plt.imshow(image)

In [ ]:
evaluate_image = [image]
                
images = np.array(evaluate_image, dtype = 'float32')

In [ ]:
print (images.shape)

In [ ]:
model_scratch.predict(images)

In [ ]:
# {'mountain': 0, 'street': 1, 'glacier': 2, 'buildings': 3, 'sea': 4, 'forest': 5}
pred_labels = np.argmax(model_scratch.predict(images), axis = 1)
print(pred_labels)

In [ ]:
for class_name in class_names_label:
    if class_names_label[class_name] == pred_labels:
        print (class_name)

In [ ]:
predictions = model_scratch.predict(test_images)
pred_labels = np.argmax(predictions, axis = 1)

In [ ]:
CM = confusion_matrix(test_labels, pred_labels)
print (CM)
ax = plt.axes()
sn.heatmap(CM, annot=True, 
           annot_kws={"size": 10}, 
           xticklabels=class_names, 
           yticklabels=class_names, ax = ax)
plt.show()

In [ ]:
# {'mountain': 0, 'street': 1, 'glacier': 2, 'buildings': 3, 'sea': 4, 'forest': 5}
for idx,image in enumerate(test_images):
    if (pred_labels[idx] == 2) and (test_labels[idx] == 4):
        print (idx)
        plt.imshow(test_images[idx])
        print (test_labels)
        plt.show()
        input()
    

Transfer Learning

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

model_vgg = VGG16(weights='imagenet', include_top=False)
model_vgg.summary()

In [ ]:
train_learned_features = model_vgg.predict(train_images)
test_learned_features = model_vgg.predict(test_images)

In [ ]:
number_train, h, w, d = train_learned_features.shape
number_test, h, w, d = test_learned_features.shape
length_of_features = h * w * d
print (h,w,d)

print(length_of_features)

In [ ]:
from sklearn import decomposition

pca = decomposition.PCA(n_components = 2)

X = train_learned_features.reshape((number_train, h*w*d))
pca.fit(X)

pc_scores = pca.transform(X) 
pc1 = pc_scores[:,0]
pc2= pc_scores[:,1]

In [ ]:
plt.subplots(figsize=(20,20))

for idx, class_name in enumerate(class_names):
    plt.scatter(pc1[train_labels == idx][:1000], pc2[train_labels == idx][:1000], label = class_name, alpha=0.4)
plt.legend()
plt.title("PCA Projection")
plt.show()

In [ ]:
model_vgg_weight = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (h, w, d)),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

In [ ]:
model_vgg_weight.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

history2 = model_vgg_weight.fit(train_learned_features, train_labels, batch_size=128, epochs=20, validation_split = 0.2)

In [ ]:
test_loss = model_vgg_weight.evaluate(test_learned_features, test_labels)

In [ ]:
image = cv2.imread("../Data/Evaluate/moun_1.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, IMAGE_SIZE)
plt.imshow(image)


evaluate_image = [image]
images = np.array(evaluate_image, dtype = 'float32')
eval_images_learned_features = model_vgg.predict(images)
model_vgg_weight.predict(eval_images_learned_features)

pred_labels = np.argmax(model_vgg_weight.predict(eval_images_learned_features), axis = 1)
print(pred_labels)
for class_name in class_names_label:
    if class_names_label[class_name] == pred_labels:
        print (class_name)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3


pre_trained_model = InceptionV3(input_shape = (299, 299, 3), 
                                include_top = True,weights = "imagenet")

pre_trained_model.summary()